# NYC Crime Data Cleaning

In [42]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [43]:
path = 'C:/Users/Zaca/Documents/Datasets/nyc/'

In [44]:
# To save memory:
# I have already pre-preared a text file containing the columns I think are most relevant.
selected_cols = pd.read_csv(path + 'crime_selected_cols.txt')
selected_cols

,name,description,rename
0,CMPLNT_FR_DT,Exact date of occurrence for the reported event,date
1,CMPLNT_FR_TM,Exact time of occurrence for the reported event,time
2,ADDR_PCT_CD,The precinct in which the incident occurred,precinct
3,KY_CD,Three digit offense classification code,class_code
4,OFNS_DESC,Description of offense corresponding with key ...,description
5,LAW_CAT_CD,Level of offense: felony misdemeanor violation,level
6,BORO_NM,The name of the borough in which the incident ...,borough
7,PREM_TYP_DESC,Specific description of premises (grocery stor...,premises
8,Lat_Lon,Geospatial Location Point (latitude and Longit...,geo


In [45]:
crime = pd.read_csv(path + 'nypd_historic.csv', usecols=selected_cols['name'])

In [46]:
# change column names
crime.columns = selected_cols['rename']

In [47]:
# re-checking the size of our dataset
crime.shape

(6847944, 9)

In [48]:
# look at dtypes
crime.dtypes

rename
date            object
time            object
precinct       float64
class_code       int64
description     object
level           object
borough         object
premises        object
geo             object
dtype: object

In [49]:
# change to appropriate dtypes
crime.date = pd.to_datetime(crime.date, errors='coerce')

In [50]:
crime.time = pd.to_datetime(crime.time, errors='coerce').dt.hour

In [51]:
# I have tons of data, might as well just drop nas.
crime.isna().sum()
crime.dropna(inplace=True)

In [52]:
crime.time = crime.time.astype('int64')

In [53]:
# clean / filter data by complete years
crime = crime[(crime.date > '01-01-2007') & (crime.date < '01-01-2019')]

In [54]:
# transform precinct column to int
crime['precinct'] = crime.precinct.astype('int64')

In [55]:
# lets clean up the categorical data
# get top crimes
crime['description'].value_counts()[:25].sum()/crime.shape[0]

0.9845345980221958

In [56]:
# decrease the number of crime categories by only taking the top 25 crimes
# taking the top 25 already includes 98% of data.
top_crimes = list(crime['description'].value_counts()[:25].index)
crime = crime.loc[crime['description'].isin(top_crimes)]

In [57]:
crime.premises.value_counts()[:35].sum()/crime.shape[0]

0.9751700758695472

In [58]:
# get top premises
top_premises = crime.premises.value_counts()[:35].index
crime = crime.loc[crime['premises'].isin(top_premises)]

In [59]:
crime.sort_values('date')

rename,date,time,precinct,class_code,description,level,borough,premises,geo
2852631,2007-01-02,19,60,107,BURGLARY,FELONY,BROOKLYN,RESIDENCE-HOUSE,"(40.580328925, -74.00746422)"
2228914,2007-01-02,15,40,352,CRIMINAL TRESPASS,MISDEMEANOR,BRONX,RESIDENCE - APT. HOUSE,"(40.807513876, -73.917016572)"
518119,2007-01-02,12,1,109,GRAND LARCENY,FELONY,MANHATTAN,GYM/FITNESS FACILITY,"(40.707321602, -74.010966266)"
1937010,2007-01-02,18,112,109,GRAND LARCENY,FELONY,QUEENS,STREET,"(40.725310425, -73.844377945)"
2334034,2007-01-02,17,14,107,BURGLARY,FELONY,MANHATTAN,DEPARTMENT STORE,"(40.750430768, -73.989282176)"
...,...,...,...,...,...,...,...,...,...
3831873,2018-12-31,13,110,341,PETIT LARCENY,MISDEMEANOR,QUEENS,FAST FOOD,"(40.74710919000006, -73.88384204599998)"
3831878,2018-12-31,18,88,110,GRAND LARCENY OF MOTOR VEHICLE,FELONY,BROOKLYN,PARKING LOT/GARAGE (PUBLIC),"(40.69022360400004, -73.96027858599997)"
3831882,2018-12-31,15,60,351,CRIMINAL MISCHIEF & RELATED OF,MISDEMEANOR,BROOKLYN,STREET,"(40.57238821200008, -73.992625934)"
3346201,2018-12-31,3,84,113,FORGERY,FELONY,BROOKLYN,STREET,"(40.694821241, -73.983265199)"


In [60]:
crime.to_csv(path + 'nypd_historic_07-18.csv')